<a href="https://colab.research.google.com/github/anshuman1808/Single-Price-Grid/blob/main/ProtBERT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ProtBERT** **Preprocessing**

1: Upload the FASTA File

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving uniprotkb_AND_reviewed_true_AND_model_o_2025_04_07.fasta to uniprotkb_AND_reviewed_true_AND_model_o_2025_04_07.fasta


2:Installing the Required Packages

In [ ]:
!pip install transformers biopython torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

 3: Full Preprocessing Code

In [ ]:
from Bio import SeqIO
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset

# Load ProtBERT tokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)

# Step 1: Parse FASTA
def parse_fasta(fasta_path):
    sequences = []
    for record in SeqIO.parse(fasta_path, "fasta"):
        sequence = str(record.seq)
        if set(sequence).issubset(set("ACDEFGHIKLMNPQRSTVWY")):  # Amino acids
            sequences.append(sequence)
    return sequences

# Step 2: Preprocess (add spaces)
def preprocess_sequence(seq):
    return ' '.join(list(seq))

# Step 3: Tokenize
def tokenize_sequences(sequences, tokenizer, max_length=512):
    preprocessed = [preprocess_sequence(seq) for seq in sequences]
    return tokenizer(
        preprocessed,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=max_length
    )

# Step 4: PyTorch Dataset
class ProteinSequenceDataset(Dataset):
    def __init__(self, sequences, tokenizer):
        self.data = tokenize_sequences(sequences, tokenizer)

    def __len__(self):
        return self.data['input_ids'].shape[0]

    def __getitem__(self, idx):
        return {
            'input_ids': self.data['input_ids'][idx],
            'attention_mask': self.data['attention_mask'][idx]
        }

# === Load and Process ===
fasta_file = "uniprotkb_AND_reviewed_true_AND_model_o_2025_04_07.fasta"
sequences = parse_fasta(fasta_file)

# Optional: reduce size for testing
sequences = sequences[:100]

# Build dataset
dataset = ProteinSequenceDataset(sequences, tokenizer)

# Check one sample
sample = dataset[0]
print("Input IDs shape:", sample['input_ids'].shape)
print("Attention mask shape:", sample['attention_mask'].shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

Input IDs shape: torch.Size([512])
Attention mask shape: torch.Size([512])


4: Visualize Tokens

In [ ]:
# View tokens
tokens = tokenizer.convert_ids_to_tokens(sample['input_ids'])
print(tokens)


['[CLS]', 'M', 'G', 'L', 'E', 'A', 'L', 'V', 'P', 'L', 'A', 'M', 'I', 'V', 'A', 'I', 'F', 'L', 'L', 'L', 'V', 'D', 'L', 'M', 'H', 'R', 'H', 'Q', 'R', 'W', 'A', 'A', 'R', 'Y', 'P', 'P', 'G', 'P', 'L', 'P', 'L', 'P', 'G', 'L', 'G', 'N', 'L', 'L', 'H', 'V', 'D', 'F', 'Q', 'N', 'T', 'P', 'Y', 'C', 'F', 'D', 'Q', 'L', 'R', 'R', 'R', 'F', 'G', 'D', 'V', 'F', 'S', 'L', 'Q', 'L', 'A', 'W', 'T', 'P', 'V', 'V', 'V', 'L', 'N', 'G', 'L', 'A', 'A', 'V', 'R', 'E', 'A', 'M', 'V', 'T', 'R', 'G', 'E', 'D', 'T', 'A', 'D', 'R', 'P', 'P', 'A', 'P', 'I', 'Y', 'Q', 'V', 'L', 'G', 'F', 'G', 'P', 'R', 'S', 'Q', 'G', 'V', 'I', 'L', 'S', 'R', 'Y', 'G', 'P', 'A', 'W', 'R', 'E', 'Q', 'R', 'R', 'F', 'S', 'V', 'S', 'T', 'L', 'R', 'N', 'L', 'G', 'L', 'G', 'K', 'K', 'S', 'L', 'E', 'Q', 'W', 'V', 'T', 'E', 'E', 'A', 'A', 'C', 'L', 'C', 'A', 'A', 'F', 'A', 'D', 'Q', 'A', 'G', 'R', 'P', 'F', 'R', 'P', 'N', 'G', 'L', 'L', 'D', 'K', 'A', 'V', 'S', 'N', 'V', 'I', 'A', 'S', 'L', 'T', 'C', 'G', 'R', 'R', 'F', 'E', 'Y', 'D', 